In [1]:
#CASE 1. 고객별 상품 빈도 파악
#CASE 2. 특정고객 구매 예측

In [2]:
#CASE 1 : 고객별 상품 빈도 파악
from jbfg.pandas_utils import *
schema = 'l2_kjbk_extnl'

In [3]:
#고객별 상품 빈도 파악, testcase : cstno,34552
sql = 'select order_id, product_id, dense_rank() over(order by order_id) AS rnk from train_x_past where cstno ="34552" order by order_id'
df1=get_df(sql,schema)

sql2 = 'select product_id, count(*) from train_x_past where cstno ="34552" group by product_id order by product_id'
df2 = get_df(sql2,schema)

In [4]:
#동일한 주문번호에 속한 상품끼리 묶기
df1['order_id'] = df1['order_id'].astype('int')
df_tmp = df1[['rnk', 'order_id','product_id']]
print(df_tmp)

     rnk  order_id product_id
0      1   1051978      35951
1      2   1061517      47626
2      2   1061517      49683
3      3   1082161      27521
4      3   1082161      35221
..   ...       ...        ...
261   76    898217      33000
262   76    898217      18465
263   77    999996      49683
264   77    999996      24852
265   77    999996      18465

[266 rows x 3 columns]


In [5]:
#인덱스생성
#for index, value in enumerate(df_tmp, start=1) :
#    print(index, value)

df_tmp_arr=[[]for i in range(78)]
num = 0
for i in df_tmp['product_id'] :
    df_tmp_arr[df_tmp['rnk'][num]].append(i)
    num += 1

df_tmp_arr.pop(0) #첫번쨰 빈데이터 제거
num=0
for i in df_tmp_arr :
    df_tmp_arr[num] = list(set(df_tmp_arr[num]))
    num+=1

print(df_tmp_arr)

[['35951'], ['49683', '47626'], ['27521', '35221'], ['35951', '35221', '24852'], ['22935', '35221', '46667', '34358'], ['18465', '35221'], ['33000'], ['18465', '27521'], ['35221', '21616', '24184'], ['18465', '35221', '34358', '30391', '10749', '5450', '21616'], ['18465', '4605', '34358', '48679', '21616', '28204'], ['35221', '24852', '28204'], ['46906'], ['18465'], ['18465'], ['21903', '49683', '24184', '5450', '39928'], ['5450', '18465'], ['47626', '35221', '33000', '21137', '49683', '9076'], ['10749', '35951', '24964'], ['18465', '47626', '35221', '16759', '34358', '46667', '22935', '28204'], ['8518', '47209', '24184', '21903', '8424'], ['5450', '24184', '21903', '44359'], ['47626', '35221', '34358', '47209', '21903', '49683', '46667'], ['35951', '47626', '35221', '33000'], ['18465', '35221', '46667', '34358'], ['35221'], ['49683', '47626', '44359'], ['49683', '18465', '34358'], ['18465', '47626', '4605', '44359', '49683', '19057', '21616'], ['35221', '33000'], ['18465', '35221', '3

In [6]:
#CASE 1 종료
#향 후 진행방향 
#1. 전체 고객에 대한 동일 주문번호에 속한 상품끼리 묶기
#2. 동일 주문번호별 입력가능한 분석모델 생성, 검증

In [7]:
##########################################

In [8]:
##########################################

In [9]:
#CASE 2 : 특정고객 구매 예측
#머신러닝을 통한 모델 생성
#Automatic scikit-learn 사용

In [10]:
#import
#from jbfg.pandas_utils import *
from autosklearn.classification import AutoSklearnClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
#import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#import os
#import random

#DB Connection
schema = 'l2_kjbk_extnl'

#seed
#현 단계 사용x, 데이터 전처리를 위해 향후 사용
#def seed_eveything(seed):
#    random.seed(seed)
#    os.environ['PYTHONHASHSEED'] = str(seed)
#    np.random.seed(seed)

In [11]:
#DataFrame
#1.train_x
sql = 'select order_number, week_day, order_hour, case when order_interval = "" then "0" else SUBSTR(order_interval,1 ,(INSTR(order_interval, ".")-1)) end as order_interval, add_order, Is_reordered from train_x_past'
train_x = get_df(sql,schema)
train_x = train_x.astype('int')

#2.train_y
sql = 'select product_id from train_x_past'
train_y = get_df(sql,schema)

#3.test_x
#고객별 예측률 파악, 129332
sql = 'select order_number, week_day, order_hour, case when order_interval = "" then "0" else TRIM(SUBSTR(order_interval,1 ,(INSTR(order_interval, ".")-1))) end as order_interval, add_order, Is_reordered from train_x_current where cstno ="129332"'
test_x = get_df(sql,schema)
test_x = test_x.astype('int')
#test_x = test_x.astype('catagory')
#test_x = test_x.astype({'is_reordered':'bool'})

In [ ]:
#Auto-Sklearn
#time_left_for_this_task, per_run_time_limit별로 정롹도 측정 필요
#간헐적 커널 종료현상발생 -> 입력변수 확인필요
automl = AutoSklearnClassifier(time_left_for_this_task = 300, per_run_time_limit=20, n_jobs=-1) 
automl.fit(train_x, train_y)

predictions = automl.predict(test_x)

In [ ]:
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
print(predictions)

In [ ]:
#Merge train_x_past, train_y 
sql = 'SELECT * FROM train_y'
df_train_y = get_df(sql, schema)
df_train_y.tail()

sql = 'SELECT * FROM train_x_current'
df_train_x_current = get_df(sql, schema)
df_train_x_current.tail()

#데이터병합
current_merge = pd.merge(df_train_x_current, df_train_y, left_index=True, right_index=True, how='left')
current_merge

In [ ]:
df_x_merge[df_x_merge['cstno']=='129332']

In [ ]:
#CASE 2 종료
#향 후 진행방향 
#1. 데이터 전처리 필요, 데이터 타입 변경에 따른 정확도 분석
#2. AutoSklearnClassifier의 변수 변경 등 필요